In [ ]:
import random, time, cv2
from matplotlib import pyplot as plt
from pydnet import darknet

random.seed(3)
input_file = './dog.jpg'

config_file = './yolov4-tiny.cfg'
data_file = './coco.data'
weights_file = './yolov4-tiny.weights'

thresh = 0.5

In [ ]:
!wget -nc https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [ ]:
# crashes the kernel if any of the files cannot be found
network, class_names, class_colors = darknet.load_network(    
    config_file,
    data_file,
    weights_file,
)

width = darknet.network_width(network)    
height = darknet.network_height(network)    
darknet_image = darknet.make_image(width, height, 3)    

image =  cv2.imread(input_file)
height_orig, width_orig, channels_orig = image.shape
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
image_resized = cv2.resize(image_rgb, (width, height),    
                           interpolation=cv2.INTER_LINEAR)    

darknet.copy_image_from_bytes(darknet_image, image_resized.tobytes())    

start = time.time()
detections = darknet.detect_image(network, class_names, darknet_image, thresh=thresh)
stop = time.time()


darknet.free_image(darknet_image)    
boxed = darknet.draw_boxes(detections, image_resized, class_colors)
image_boxes = cv2.cvtColor(boxed, cv2.COLOR_BGR2RGB)

darknet.print_detections(detections, True)

print(f'Time: {round(stop-start, 4)} s -- FPS: {int(1/(stop-start))}')

In [ ]:
rcol = lambda: (random.randint(0, 255),
               random.randint(0, 255),
               random.randint(0, 255))

def get_scale_factor(orig, rescaled=None):
    if rescaled is None:
        return (1,1)
    o_height, o_width, o_channels = orig.shape
    r_height, r_width, r_channels = rescaled.shape
    fx = o_width/r_width
    fy = o_height/r_height
    return fx, fy

def draw_bboxes(img, dets, resized_img=None):

    font = cv2.FONT_HERSHEY_DUPLEX
    font_scale = 1
    font_thickness = 1
    text_color = (255, 255, 255)

    for det in dets:
        name, conf, bbox = det
        text = f'{name}: {conf}'
        color = rcol()
    
        points = darknet.bbox2points(bbox)
        sx, sy, ex, ey = points

        fx, fy = get_scale_factor(img, resized_img)
        start, end = (int(sx*fx), int(sy*fy)), (int(ex*fx), int(ey*fy))
        cv2.rectangle(img, start, end, color, 2)

        x, y = start
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_w, text_h = text_size
        cv2.rectangle(img, (x, y-text_h), (x+text_w, y), color, -1)
        cv2.putText(img, text, (x, y+font_scale-1), font, font_scale, text_color, font_thickness)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20,20))

axes[0].imshow(image)
axes[1].imshow(image_boxes)

tmp = image.copy()
draw_bboxes(tmp, detections, image_resized)
axes[2].imshow(tmp)

plt.tight_layout()
plt.show()